# 🧪 Prompt 优化实验 (Prompt Optimization)

> 基于 `ITERATION_GUIDE.md` 的 Prompt 优化实验 Notebook。

**实验目标**:
1. **准确性提升**: 解决复杂查询下的意图识别准确性问题（特别是相对日期和多步推理）。
2. **成本降低**: 减少 Input Token 消耗 20%。
3. **幻觉抑制**: 减少 Agent 编造工具参数的情况。

**实验内容**:
- **实验一**: 动态上下文注入优化 (Dynamic Context)
- **实验二**: Few-Shot 示例精简与重构
- **实验三**: 思维链 (CoT) 引导增强
- **对比测试**: 多场景准确性与成本分析

In [11]:
# 1. 环境配置与基准数据加载
import sys
import os
import json
import pandas as pd
from datetime import datetime

# 添加项目根目录到 path
sys.path.append("..")

# 导入核心模块
from agent_logic import _get_system_prompt, run_agent
from core.llm_client import get_client_for_model

# 加载基准测试数据
BASELINE_FILE = "benchmarks/v1.1.0_baseline.json"
if os.path.exists(BASELINE_FILE):
    with open(BASELINE_FILE, "r") as f:
        baseline_data = json.load(f)
    print(f"✅ 已加载基准数据: {len(baseline_data)} 条记录")
else:
    print("⚠️ 未找到基准数据，将建立新的基准。")
    baseline_data = {}

# 设置测试模型
TEST_MODEL = "qwen3-max"
print(f"🧪 测试模型: {TEST_MODEL}")

✅ 已加载基准数据: 4 条记录
🧪 测试模型: qwen3-max


In [ ]:
# 2. 当前 Prompt 架构分析
current_prompt = _get_system_prompt()
print("=== 当前 System Prompt (v1.2.0) ===")
print(current_prompt[:500] + "...\n[省略中间部分]\n..." + current_prompt[-300:])  # 仅显示首尾，避免输出过长
print("=====================================")

# 分析 Token 消耗 (估算)
# 简单估算: 1汉字 ~ 2 tokens, 1英文单词 ~ 1.3 tokens
def estimate_tokens(text):
    """估算文本的 Token 数量（粗略）"""
    chinese_chars = len([c for c in text if '\u4e00' <= c <= '\u9fff'])
    other_chars = len(text) - chinese_chars
    return chinese_chars * 2 + other_chars * 0.5

current_tokens = estimate_tokens(current_prompt)
print(f"📏 当前 System Prompt 长度: {len(current_prompt)} 字符")
print(f"🎫 估算 Token 数: ~{int(current_tokens)}")
print(f"📊 优化目标: 减少 20% -> ~{int(current_tokens * 0.8)} tokens")

=== 当前 System Prompt (v1.1.2) ===
你是一名专业的量化金融分析师助手。你的任务是协助用户获取金融市场数据，计算技术指标，并生成可视化图表来分析市场趋势。

**重要时间信息**: 今天是 2025年12月14日。
当用户提到"最近X天/月"、"近期"、"当前"等相对时间词时，请基于 2025年12月14日 来计算日期范围。

你的能力包括：
1. 获取 A 股和 ETF 的历史行情数据
2. 计算技术指标（MA、MACD、RSI、布林带等）
3. 生成专业的 K 线图和指标图表
4. 基于技术指标提供客观的市场分析

你的回答应当：
- 数据驱动，基于实际的市场数据
- 客观中立，不做主观预测
- 优先展示可视化分析结果
- 清晰解释技术指标的含义

你可以使用以下工具：

**fetch_stock_data**
获取 A 股历史数据。
参数：
- symbol: 股票代码（例如 "600519" 表示贵州茅台）
- days: 获取最近 N 天的数据（整数，推荐使用此参数）
  * 如果用户说"最近两个月"，请传递 60
  * 如果用户说"近一周"，请传递 7
  * 如果用户说"三个月"，请传递 90
- start_date: 开始日期（格式：YYYYMMDD，可选）
- end_date: 结束日期（格式：YYYYMMDD，可选）

**推荐**：优先使用 `days` 参数，系统会自动计算对应的日期范围（从今天往前推）。

**fetch_etf_data**
获取 ETF 历史数据。
参数：
- symbol: ETF 代码（例如 "510300" 表示沪深300ETF）
- days: 获取最近 N 天的数据（整数，推荐使用此参数）
  * 如果用户说"最近两个月"，请传递 60
  * 如果用户说"近一周"，请传递 7
  * 如果用户说"三个月"，请传递 90
- start_date: 开始日期（格式：YYYYMMDD，可选）
- end_date: 结束日期（格式：YYYYMMDD，可选）

**推荐**：优先使用 `days` 参数，系统会自动计算对应的日期范围（从今天往前推）。

**analyze_and_plot**
分析数据并生成图表。
参数：
- data_id: 数据标识符（由前面的 fetch

In [12]:
# 3. 实验一：动态上下文注入优化 (Dynamic Context)
# 目标：确保“上个季度”、“今年年初”等相对时间词能被 100% 准确解析

def get_optimized_system_prompt_v1():
    """
    优化版 System Prompt v1: 增强日期上下文
    """
    current_date = datetime.now()
    date_str = current_date.strftime("%Y年%m月%d日")
    weekday_str = ["周一", "周二", "周三", "周四", "周五", "周六", "周日"][current_date.weekday()]
    
    # 计算常用相对日期
    one_month_ago = (current_date - pd.Timedelta(days=30)).strftime("%Y%m%d")
    year_start = current_date.strftime("%Y0101")
    
    return f"""你是一名专业的量化金融分析师助手。

**当前时间上下文**:
- 今天是: {date_str} ({weekday_str})
- 年初至今: {year_start} 至 {current_date.strftime("%Y%m%d")}
- 近一个月: {one_month_ago} 至 {current_date.strftime("%Y%m%d")}

当用户查询相对时间（如"最近"、"今年"）时，请参考上述时间上下文。
优先使用 `days` 参数调用工具，让系统自动处理日期计算。

... (其余部分保持不变或精简) ...
"""

print(get_optimized_system_prompt_v1())

你是一名专业的量化金融分析师助手。

**当前时间上下文**:
- 今天是: 2025年12月14日 (周日)
- 年初至今: 20250101 至 20251214
- 近一个月: 20251114 至 20251214

当用户查询相对时间（如"最近"、"今年"）时，请参考上述时间上下文。
优先使用 `days` 参数调用工具，让系统自动处理日期计算。

... (其余部分保持不变或精简) ...



In [7]:
# 4. 实验二：Few-Shot 示例精简与重构
# 目标：移除冗余对话，保留核心模式，减少 Token

FEW_SHOT_EXAMPLES = """
示例 1 (基础查询):
User: 分析茅台最近两个月的走势
Assistant:
{
  "thought": "用户查询贵州茅台(600519)近60天走势",
  "action": "fetch_stock_data",
  "action_input": { "symbol": "600519", "days": 60 }
}

示例 2 (多步分析):
User: 对比茅台和五粮液的MACD
Assistant:
{
  "thought": "需要分别获取茅台(600519)和五粮液(000858)的数据，然后计算MACD",
  "action": "fetch_stock_data",
  "action_input": { "symbol": "600519", "days": 60 }
}
... (Agent 收到数据后) ...
{
  "thought": "已获取茅台数据，现在获取五粮液数据",
  "action": "fetch_stock_data",
  "action_input": { "symbol": "000858", "days": 60 }
}
"""

print("=== 优化后的 Few-Shot 示例 ===")
print(FEW_SHOT_EXAMPLES)

=== 优化后的 Few-Shot 示例 ===

示例 1 (基础查询):
User: 分析茅台最近两个月的走势
Assistant:
{
  "thought": "用户查询贵州茅台(600519)近60天走势",
  "action": "fetch_stock_data",
  "action_input": { "symbol": "600519", "days": 60 }
}

示例 2 (多步分析):
User: 对比茅台和五粮液的MACD
Assistant:
{
  "thought": "需要分别获取茅台(600519)和五粮液(000858)的数据，然后计算MACD",
  "action": "fetch_stock_data",
  "action_input": { "symbol": "600519", "days": 60 }
}
... (Agent 收到数据后) ...
{
  "thought": "已获取茅台数据，现在获取五粮液数据",
  "action": "fetch_stock_data",
  "action_input": { "symbol": "000858", "days": 60 }
}



In [8]:
# 5. 实验三：思维链 (CoT) 引导增强
# 目标：强制输出 Thought 步骤，提高复杂任务执行率

def get_cot_system_prompt():
    base_prompt = _get_system_prompt()
    cot_instruction = """
**思维链要求**:
在调用工具前，你必须先进行思考(Thought)，步骤如下：
1. **意图识别**: 用户想要什么？(数据获取/指标计算/图表绘制)
2. **参数提取**: 提取股票代码、时间范围（转换为 days）。
3. **工具选择**: 选择最合适的工具。
4. **缺失检查**: 如果缺少关键信息（如股票代码），请反问用户。

请严格按照 JSON 格式输出 Thought 和 Action。
"""
    return base_prompt + cot_instruction

print("=== CoT 增强指令 ===")
print(get_cot_system_prompt())

=== CoT 增强指令 ===
你是一名专业的量化金融分析师助手。你的任务是协助用户获取金融市场数据，计算技术指标，并生成可视化图表来分析市场趋势。

**重要时间信息**: 今天是 2025年12月14日。
当用户提到"最近X天/月"、"近期"、"当前"等相对时间词时，请基于 2025年12月14日 来计算日期范围。

你的能力包括：
1. 获取 A 股和 ETF 的历史行情数据
2. 计算技术指标（MA、MACD、RSI、布林带等）
3. 生成专业的 K 线图和指标图表
4. 基于技术指标提供客观的市场分析

你的回答应当：
- 数据驱动，基于实际的市场数据
- 客观中立，不做主观预测
- 优先展示可视化分析结果
- 清晰解释技术指标的含义

你可以使用以下工具：

**fetch_stock_data**
获取 A 股历史数据。
参数：
- symbol: 股票代码（例如 "600519" 表示贵州茅台）
- days: 获取最近 N 天的数据（整数，推荐使用此参数）
  * 如果用户说"最近两个月"，请传递 60
  * 如果用户说"近一周"，请传递 7
  * 如果用户说"三个月"，请传递 90
- start_date: 开始日期（格式：YYYYMMDD，可选）
- end_date: 结束日期（格式：YYYYMMDD，可选）

**推荐**：优先使用 `days` 参数，系统会自动计算对应的日期范围（从今天往前推）。

**fetch_etf_data**
获取 ETF 历史数据。
参数：
- symbol: ETF 代码（例如 "510300" 表示沪深300ETF）
- days: 获取最近 N 天的数据（整数，推荐使用此参数）
  * 如果用户说"最近两个月"，请传递 60
  * 如果用户说"近一周"，请传递 7
  * 如果用户说"三个月"，请传递 90
- start_date: 开始日期（格式：YYYYMMDD，可选）
- end_date: 结束日期（格式：YYYYMMDD，可选）

**推荐**：优先使用 `days` 参数，系统会自动计算对应的日期范围（从今天往前推）。

**analyze_and_plot**
分析数据并生成图表。
参数：
- data_id: 数据标识符（由前面的 fetch 工具返回）
- chart_ty

In [ ]:
# 6. 多场景准确性对比测试
# 构建测试集
test_cases = [
    {"query": "分析茅台最近两个月的走势", "expected_days": 60, "type": "relative_date"},
    {"query": "看看平安银行今年的表现", "expected_days": None, "type": "year_to_date"}, # 应该计算从1月1日至今的天数
    {"query": "分析 600519 和 000858 的 RSI", "expected_tools": 2, "type": "multi_symbol"},
]

def run_test(prompt_version, cases):
    results = []
    for case in cases:
        print(f"Testing: {case['query']}...")
        # 这里模拟运行 Agent，实际需要 hook run_agent 的 prompt 参数
        # result = run_agent(case['query'], system_prompt=prompt_version)
        # results.append(evaluate(result, case))
        pass
    return results

# 运行测试 (伪代码，需完善 run_agent 接口支持传入 prompt)
print("🚀 开始运行对比测试...")

In [10]:
# 7. Token 消耗与成本效益分析

def calculate_cost_saving(original_tokens, optimized_tokens):
    saving = (original_tokens - optimized_tokens) / original_tokens * 100
    return saving

# 假设数据
original_avg_tokens = 1500
optimized_avg_tokens = 1100

saving = calculate_cost_saving(original_avg_tokens, optimized_avg_tokens)
print(f"💰 预计 Token 节省: {saving:.1f}%")

# 绘制对比图
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.bar(["Original", "Optimized"], [original_avg_tokens, optimized_avg_tokens], color=['gray', 'green'])
plt.title("Average Token Consumption per Turn")
plt.ylabel("Tokens")
plt.show()

💰 预计 Token 节省: 26.7%


💰 预计 Token 节省: 26.7%


/var/folders/m7/3jmn7pzs08d_p3g0s645cmnr0000gn/T/ipykernel_81946/3649479068.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
# 8. 导出优化后的 Prompt 配置
# 将最佳实践保存到文件

OPTIMIZED_PROMPT_TEMPLATE = """
# ... 最终优化的 Prompt 内容 ...
"""

output_path = "outputs/optimized_prompt_v1.2.0.py"
with open(output_path, "w") as f:
    f.write(f'SYSTEM_PROMPT_V1_2_0 = """{OPTIMIZED_PROMPT_TEMPLATE}"""')

print(f"✅ 优化后的 Prompt 已导出至: {output_path}")
print("📝 请记得更新 optimization/experiments/experiment_log.md 记录实验结果")